In [9]:
import logging
logging.basicConfig(level=logging.NOTSET)

In [13]:
from dataset import UserRatingsDataset, Dataset
from torch.utils.data import DataLoader

cutoff = lambda x: 1.0 if x >= 3.5 else 0.0 # function for generating implicit ratings

dataset = UserRatingsDataset("../data/ml-10m/ratings.csv",threshold=20, rating_function=cutoff)
train, valid, test = dataset.tvt_datasets()

train = DataLoader(train, batch_size=100, num_workers=1)
valid = DataLoader(valid, batch_size=100, num_workers=1)
test = DataLoader(test, batch_size=100, num_workers=1)

DEBUG:data.dataset:Loaded pandas dataframe into memory
DEBUG:data.dataset:Applying filter strategies to dataset
DEBUG:data.dataset:Finished applying filter strategies to dataset
DEBUG:data.dataset:Finished cleaning
INFO:data.dataset:Created surprise dataset
INFO:data.dataset:Finished building surprise trainset


In [14]:
from model import Model
model = Model(dataset.n_items, 200)

In [15]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping

trainer = pl.Trainer(
    max_epochs=200,
    log_every_n_steps=10,
    callbacks=[EarlyStopping(monitor="ndcg@100", mode="max", patience=20)],
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [16]:
trainer.fit(model, train, valid)

INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type    | Params
------------------------------------
0 | drop    | Dropout | 0     
1 | encoder | Encoder | 6.6 M 
2 | decoder | Decoder | 6.5 M 
------------------------------------
13.2 M    Trainable params
0         Non-trainable params
13.2 M    Total params
52.739    Total estimated model params size (MB)
DEBUG:fsspec.local:open file: /home/nikola/projects/deepRec/vae/lightning_logs/version_5/hparams.yaml


Sanity Checking: 0it [00:00, ?it/s]

/home/nikola/projects/deepRec/env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/home/nikola/projects/deepRec/env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:  48%|████▊     | 271/560 [00:25<00:27, 10.44it/s, v_num=5]

/home/nikola/projects/deepRec/env/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
DEBUG:fsspec.local:open file: /home/nikola/projects/deepRec/vae/lightning_logs/version_5/hparams.yaml


In [ ]:
trainer.test(model, test)